Build a graph from database data

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
from matplotlib import pyplot as plt

In [ ]:
# etl

pg_host = 'localhost'
pg_port = 5432
pg_user = 'postgres'
pg_pass = 'postgres'
pg_db = 'pgdb'

engine = create_engine(f'postgresql+psycopg2://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}')
pg_conn = engine.connect()

etl = pd.read_csv('./data/csv/gapminder.csv')
etl.to_sql('gapminder', pg_conn, if_exists='replace')
pg_conn.commit()

In [ ]:
# select data

data = pd.read_sql_query(sql=text('select * from gapminder'), con=pg_conn)
print(data)

pg_conn.close()

analysis

In [ ]:
latest = data.year.max()
latest

In [ ]:
earliest = data.year.min()
earliest

In [ ]:
data.year.unique()

In [ ]:
# only latest year

sel_year = data[data.year == latest]
sel_year

In [ ]:
continent = sel_year.groupby('continent')
continent.mean().lifeExp.plot(legend=True, title=f'Life Expectancy in {latest}')
plt.savefig('./results/lifeExp.png')
plt.show()
continent.pop.sum().plot(legend=True, title=f'Population in {latest}')
plt.savefig('./results/population.png')
plt.show()
continent.gdpPercap.sum().plot(legend=True, title=f'GDP per Capita in {latest}')
plt.savefig('./results/gdp.png')
plt.show()